Hi. In this project, I'm going to analyze IPL Data to find out the most valuable players for fantasy cricket competitions. For this, I have used the points system as specified by the popular fantasy platform 'Dream 11'.

Note: The points calculated aren't exactly correct as I have not included points from Run Outs, and Economy+Strike Rates for particular innings, as that would require innings-by-innings data. The latter part is negligible anyways and the run-out points would create at most a difference of 1-2 points per match for good fielders. All major criterias have been taken into account so overall you can consider this a pretty accurate analysis.

Let's Begin by doing the necessary imports:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None

Our first step is to collect the data. I have gathered the data from ESPN Cricinfo's pages for aggregated batting and bowling stats of players of different teams. First, the batting stats:

In [2]:
cskbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4343;type=trophy')[0]
dcbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4347;type=trophy')[0]
ddbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4344;type=trophy')[0]
glbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=5845;type=trophy')[0]
kxipbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4342;type=trophy')[0]
ktkbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4788;type=trophy')[0]
kkrbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4341;type=trophy')[0]
mibat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4346;type=trophy')[0]
pwibat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4787;type=trophy')[0]
rrbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4345;type=trophy')[0]
rpsbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=5843;type=trophy')[0]
rcbbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4340;type=trophy')[0]
srhbat=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=5143;type=trophy')[0]

Similarly, the bowling stats:

In [3]:
cskbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4343;type=trophy')[0]
dcbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4347;type=trophy')[0]
ddbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4344;type=trophy')[0]
glbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=5845;type=trophy')[0]
kxipbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4342;type=trophy')[0]
ktkbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4788;type=trophy')[0]
kkrbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4341;type=trophy')[0]
mibowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4346;type=trophy')[0]
pwibowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4787;type=trophy')[0]
rrbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4345;type=trophy')[0]
rpsbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=5843;type=trophy')[0]
rcbbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=4340;type=trophy')[0]
srhbowl=pd.read_html('https://stats.espncricinfo.com/ci/engine/records/averages/bowling.html?id=117;team=5143;type=trophy')[0]

Here's an example of what the raw data looks like:

In [4]:
cskbowl

,Player,Span,Mat,Inns,Overs,Mdns,Runs,Wkts,BBI,Ave,Econ,SR,4,5,Ct,St
0,P Amarnath,2008-2008,6,6,22.0,0,236,7,2/29,33.71,10.72,18.8,0,0,2,0
1,S Anirudha,2008-2013,19,-,-,-,-,-,-,-,-,-,-,-,11,0
2,KB Arun Karthik,2010-2010,1,-,-,-,-,-,-,-,-,-,-,-,0,0
3,R Ashwin,2009-2015,97,94,337.4,4,2180,90,3/16,24.22,6.45,22.5,0,0,20,0
4,KM Asif,2018-2018,2,2,6.0,0,75,3,2/43,25.00,12.50,12.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,S Vidyut,2008-2008,9,1,2.0,0,22,1,1/22,22.00,11.00,12.0,0,0,1,0
72,M Vijay,2009-2020,68,-,-,-,-,-,-,-,-,-,-,-,33,0
73,SR Watson,2018-2020,33,11,28.0,0,251,6,2/29,41.83,8.96,28.0,0,0,10,0
74,DJ Willey,2018-2018,3,3,10.0,0,95,2,1/24,47.50,9.50,30.0,0,0,2,0


Now that we have our team-wise data separated, lets combine it together into 2 separate dataframes for batting and bowling stats:  dfbat and dfbowl

In [5]:
dfbat=pd.concat([cskbat,dcbat,ddbat,glbat,kxipbat,ktkbat,kkrbat,mibat,pwibat,rrbat,rpsbat,rcbbat,srhbat])
dfbowl=pd.concat([cskbowl,dcbowl,ddbowl,glbowl,kxipbowl,ktkbowl,kkrbowl,mibowl,pwibowl,rrbowl,rpsbowl,rcbbowl,srhbowl])

Now we need to clean the data to obtain it in the desired form: This includes Replacing null values with 0, dropping a few columns from both dataframes and converting string to float values.

In [6]:
dfbat=dfbat.replace('-',0)
dfbowl=dfbowl.replace('-',0)
dfbat=dfbat.drop(['Span','HS','Ave','SR'],axis=1)
dfbowl=dfbowl.drop(['Span','BBI','Ave','Econ','SR'],axis=1)
dfbat=dfbat.apply(pd.to_numeric,errors='ignore')
dfbowl=dfbowl.apply(pd.to_numeric,errors='ignore')

Once we have our combined data, we group it by player. This is necessary as many players have represented multiple franchises in their career, hence there are multiple rows of data for them, which needs to be grouped together as a single row. Then, we create columns like Strike Rate(SR),Average(Ave) based off our understanding of cricket and sort the dataframe.

In [7]:
dfbat=dfbat.groupby('Player').sum()
dfbat['SR']=dfbat['Runs']*100/dfbat['BF']
dfbat['Ave']=dfbat['Runs']/(dfbat['Inns']-dfbat['NO'])
dfbat=dfbat.sort_values(by='Runs',ascending=False)

We do a similar process for the bowling dataframe.

In [8]:
dfbowl=dfbowl.groupby('Player').sum()
dfbowl['Bowl Ave']=dfbowl['Runs']/dfbowl['Wkts']
dfbowl['Econ']=dfbowl['Runs']/dfbowl['Overs']
dfbowl['Bowl SR']=dfbowl['Overs']*6/dfbowl['Wkts']
dfbowl=dfbowl.sort_values(by='Wkts',ascending=False)

This is what our combined batting dataframe looks like:

In [9]:
dfbat

,Mat,Inns,NO,Runs,BF,100,50,0,4s,6s,SR,Ave
Player,,,,,,,,,,,,
V Kohli,177,169,26,5412,4112,5,36,6,480,190,131.614786,37.846154
SK Raina,193,189,28,5368,3915,1,38,8,493,194,137.113665,33.341615
RG Sharma,189,184,28,4910,3754,1,36,12,433,194,130.793820,31.474359
DA Warner,126,126,17,4706,3305,4,44,7,458,181,142.390318,43.174312
S Dhawan,160,159,21,4579,3671,0,37,7,524,96,124.734405,33.181159
...,...,...,...,...,...,...,...,...,...,...,...,...
T Natarajan,6,0,0,0,0,0,0,0,0,0,NaN,NaN
T Mishra,1,0,0,0,0,0,0,0,0,0,NaN,NaN
S Midhun,1,0,0,0,0,0,0,0,0,0,NaN,NaN


The next step is to apply the points criteria to calculate different types of points earned during a match. This is classified into new columns in our dataframe based on Dream11 points system, eg, 1 point for each run, 2 bonus points for each six etc.

In [ ]:
dfbat['Runs Points']=dfbat['Runs']
dfbat['Boundary Bonus']=dfbat['4s']
dfbat['Six Bonus']=2*dfbat['6s']
dfbat['Half Century Bonus']=8*dfbat['50']
dfbat['Century Bonus']=16*dfbat['100']
dfbat['Dismissal For A Duck']=-2*dfbat['0']

Similarly we apply the criteria to the bowling dataframe, which contains columns for both bowling and fielding stats,eg, 25 points for each wicket, 8 points for each catch etc.

In [ ]:
dfbowl['Wickets Points']=25*dfbowl['Wkts']
dfbowl['4 Wicket Haul Bonus']=8*dfbowl['4']
dfbowl['5 Wicket Haul Bonus']=16*dfbowl['5']
dfbowl['Maiden Over Bonus']=8*dfbowl['Mdns']
dfbowl['Catch Points']=8*dfbowl['Ct']
dfbowl['Stumping Points']=12*dfbowl['St']

The next step is to combine both our batting and bowling dataframes into one master dataframe and add another criteria: 4 points for being in Starting 11. On combining, we then drop all columns not required for the analysis and retain only the points and matches columns

In [ ]:
dfbat=dfbat.reset_index()
dfbowl=dfbowl.reset_index()
dfbowl_tojoin=dfbowl.drop(['Mat','Inns','Runs'],axis=1)
df=pd.merge(dfbat,dfbowl_tojoin,on='Player')
df['Starting 11 Points']=4*df['Mat']
dfpoints=df.drop(['Inns','NO','Runs','BF','100','50','0','4s','6s','SR','Ave',
                  'Overs','Mdns','Wkts','4','5','Ct','St','Bowl Ave','Econ','Bowl SR'],axis=1)

Now we add all the different points in different categories as Total Points and find the most important metric for the analysis, Total Points Per Match (stored as Total PPM)

In [ ]:
dfpoints['Total Points']=dfpoints.drop('Mat',axis=1).sum(axis=1)
dfpoints['Total PPM']=(dfpoints['Total Points']/dfpoints['Mat']).round(2)

Now I apply a criteria to show only those players who have played a minimum of 30 matches. This roughly corresponds to the length of 2 full seasons, which is a good enough sample size to judge players' performances. The final dataframe is stored as dfbest.

In [ ]:
dffilter=dfpoints[dfpoints['Mat']>=1]
dfbest=dffilter.sort_values(by='Total PPM',ascending=False)
dfbest=dfbest.set_index('Player')

Finally, we have our results. The Fantasy points earned per match by 171 players, the most valuable 25 of which are shown below:

In [ ]:
dfbest

Wow! Some of the usual suspects but few unexpected names right up there too. 

INSIGHTS: 1)All rounders like Watson and Russell leading the charts, with Kallis(considered more of a test match player) a               surprise entry in the top 5.

2) Popular captaincy choices in fantasy leagues like Kohli, De Villiers surprisingly fetch relatively lower points,             and players like Rohit,Dhoni,Malinga and Narine don't even make it to the top 25!

Now that we're at it, let's also take a quick look at the 10 least valuable players. Unsurprisingly, most of them are no longer getting picked and have had short careers

In [ ]:
dfbest.tail(10).sort_values('Total PPM')

Let's visualize the distribution of the Total Points per match data:

In [ ]:
sns.distplot(dfbest['Total PPM'],color='black',bins=15)
print('Mean is {}, Standard Deviation is {}'.format(dfbest['Total PPM'].mean(), dfbest['Total PPM'].std()))

The distribution shows points ranging from about 16 to 57 with most of the players having points between the 30-40 range

Now let's dive a bit deeper into our analysis and classify the total points per match into Batting, Bowling and Fielding points per match (Stored as Batting PPM, Bowling PPM and Fielding PPM respectively): 

In [ ]:
dfbest['Batting PPM']=(dfbest[['Runs Points','Boundary Bonus','Six Bonus','Half Century Bonus',
                                            'Century Bonus','Dismissal For A Duck']].sum(axis=1)/dfbest['Mat']).round(2)
dfbest['Bowling PPM']=(dfbest[['Wickets Points','4 Wicket Haul Bonus','5 Wicket Haul Bonus',
                                            'Maiden Over Bonus']].sum(axis=1)/dfbest['Mat']).round(2)
dfbest['Fielding PPM']=(dfbest[['Catch Points','Stumping Points']].sum(axis=1)/dfbest['Mat']).round(2)

Lets look at the top players by batting, bowling and fielding points per match:

In [ ]:
dfbest['Batting PPM'].sort_values(ascending=False).head(10)

In [ ]:
dfbest['Bowling PPM'].sort_values(ascending=False).head(10)

In [ ]:
dfbest['Fielding PPM'].sort_values(ascending=False).head(10)

With the data classified into the respective sections, its time to visualize the points distribution for the top,say,35 players

In [ ]:
df1=dfbest[['Batting PPM','Bowling PPM','Fielding PPM']].iloc[0:35]
df1.plot.bar(stacked=True,figsize=(16,6))
plt.xticks(rotation=45)
plt.ylabel('Total Points Per Match',fontsize=12)
plt.xlabel('Player',fontsize=13)
plt.title('Most Valuable Players by Points Type',fontsize=14)
plt.tight_layout()

INSIGHTS: 1) The chart shows that its the batsmen that dominate the most at the top. We only see few proper bowlers in the top 30 and then only from positions 30-35 do we see some proper bowlers getting in.

2) Warner actually surpasses Gayle as an opening batsman (and as a fielder) but Gayle pips him on overall points based on the few(18) wickets he's taken as a part timer.

3) For players like Pant, De Kock and Gilchrist, Fielding points constitute a decent amount of total points, which is because as keepers, they get more catches and stumping oppurtunities.

4) Krunal Pandya and Stokes are two such all-rounders, who earn almost identical points in both the batting and bowling categories.

Lets take a look at the best All Rounders by plotting Total Bowling Points vs Total Batting Points per match. I've defined an All Rounder to be someone who fetches atleast 10 points per match in both disciplines. 22 players fit this definition, which are shown below:

In [ ]:
dfallround=dfbest[(dfbest['Batting PPM']>10)&(dfbest['Bowling PPM']>10)]

plt.figure(figsize=(16,8))
plt.scatter(x='Batting PPM',y='Bowling PPM',data=dfallround,c='Total PPM',cmap='viridis_r',s=dfallround['Total PPM']*2)
plt.title('Top All Rounders',fontsize=15)
plt.xlabel('Batting Points Per Match',fontsize=13)
plt.ylabel('Bowling Points Per Match',fontsize=12)
x=np.arange(10,40,5)
y=np.arange(10,32.5,2.5)
plt.xticks(ticks=x,labels=x)
plt.yticks(ticks=y,labels=y)
plt.grid(lw=0.3)
for index, row in dfallround.reset_index().iterrows():
    plt.annotate(row['Player'], xy = (row['Batting PPM'], row['Bowling PPM']),
                 xytext = (row['Batting PPM']+0.1, row['Bowling PPM']+0.2), fontsize=12)
plt.tight_layout()

INSIGHTS: This graph clearly distinguishes bowling all rounders(upper left) from batting ones(lower right) with proper all rounders lying close to the y=x diagonal line (batting points almost same as bowling points)

For the final visualization, we're going to plot all 171 players from the dfbest dataframe and look for the players who have performed most consistently over a long career. For this, I've set a criteria of at least 100 career matches while maintaining at least 45 Total Points Per Match:

In [ ]:
dffilter2=dfbest[(dfbest['Mat']>=100)&(dfbest['Total PPM']>=45)]

plt.figure(figsize=(13,7),dpi=70)
plt.scatter(x='Mat',y='Total PPM',data=dfbest,c='Total PPM',cmap='viridis_r')
plt.title('Most Consistent Players',fontsize=15)
plt.xlabel('No. of Matches',fontsize=13)
plt.ylabel('Total Points Per Match',fontsize=12)
x=np.arange(30,220,10)
y=np.arange(10,65,5)
plt.xticks(ticks=x,labels=x)
plt.yticks(ticks=y,labels=y)
plt.grid(lw=0.3)
plt.axvline(x=100, color='r',ls='--',lw=1.5)
plt.axhline(y=45, color='b',ls='--',lw=1.5)
for index, row in dffilter2.reset_index().iterrows():
    plt.annotate(row['Player'], xy = (row['Mat'], row['Total PPM']),
                 xytext = (row['Mat']+0.8, row['Total PPM']+0.4), fontsize=12)
plt.tight_layout()

INSIGHTS: 1) Watson, Gayle, Warner, Bravo, Kohli and Raina are the most overall consistent players.

2) Raina, despite having played the most matches(193), has maintained Total Points Per Match>45. The graph above shows his impact and longevity together. No wonder he is referred to as "Mr. IPL"

3) Kallis and De Villiers miss out narrowly (98 matches and 44.91 PPM respectively)

That's all for this analysis. I hope this was insightful and would help you make smarter, data driven decisions next time you select your fantasy teams. 

THANK YOU!